<a href="https://colab.research.google.com/github/raulbenitez/introAI_ICE/blob/main/PCA_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data

In [1]:
import seaborn as sns

df = sns.load_dataset('penguins') # load data
df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex'],
      dtype='object')

Remove useless variables

In [2]:
df.drop(['island','sex'],axis=1,inplace=True)
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0
3,Adelie,NaN,NaN,NaN,NaN
4,Adelie,36.7,19.3,193.0,3450.0


Remove observations with NaNs

In [3]:
df.dropna(inplace=True)
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0
4,Adelie,36.7,19.3,193.0,3450.0
5,Adelie,39.3,20.6,190.0,3650.0


Get features and class labels

In [19]:
X = df[['bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g']]
y = df['species']

PCA analysis

In [20]:
from sklearn.preprocessing import StandardScaler
XS = StandardScaler().fit_transform(X)

Aplicar PCA a dades estandarditzades

In [21]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=4).fit(XS)

In [22]:
100*pca.explained_variance_ratio_.cumsum()

array([ 68.8438781 ,  88.15679694,  97.2876946 , 100.        ])

Ens quedem amb només els 3 primers components PCA

In [23]:
Xproj = pca.transform(XS)
XPCA = Xproj[:,0:3]
XPCA.shape # nou conjunt de dades PCA en 3D

(342, 3)

Split train/test les dades PCA

In [24]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(XPCA, y,train_size=0.7)

Ajustem un model de classificació (Perceptró multicapa)

In [25]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
y_test_pred = clf.fit(Xtrain,ytrain).predict(Xtest)


Matriu de confusió

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest, y_test_pred)

array([[50,  1,  0],
       [ 1, 20,  0],
       [ 1,  0, 30]])

Comparem amb el model entrenat amb les dades 4d originals:

In [28]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,train_size=0.7)
clf1 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
y_test_pred1 = clf1.fit(Xtrain,ytrain).predict(Xtest)
confusion_matrix(ytest, y_test_pred1)

array([[42,  0,  0],
       [19,  0,  0],
       [42,  0,  0]])

In [29]:
from sklearn.neighbors import KNeighborsClassifier

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,train_size=0.7)
knn = KNeighborsClassifier(n_neighbors=5,weights='distance') #definir el modelo (parametro k)

y_test_pred2 = knn.fit(Xtrain,ytrain).predict(Xtest)
confusion_matrix(ytest, y_test_pred2)

array([[39,  4,  5],
       [ 5, 11,  3],
       [ 0,  1, 35]])